In [3]:
# Import relevant libaries
import psycopg2
import time
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
# create an Empty DataFrame object
df = pd.DataFrame()

In [4]:
def process_youtubedata_file(cur, conn, filepath,df1): 
    """
        This function reads one JSON file and read information of videos and youtuber data and saves into video_data and youtuber_data
        Arguments:
        cur: Database Cursor
        filepath: location of JSON files
        Return: None
    """
    # open JSON file
    #print(filepath)
   # df1 = pd.read_json(filepath)
    #df.fillna('')
    
    # ---------insert youtuber record----------
    # write your code here that reads youtuber data from JSON file and insert it into Youtubers_dim table 

    # write your code here
    # youtuber_data = df1[['youtuber_id', 'youtuber_name', 'youtuber_location', 'youtuber_latitude', 'youtuber_longitude']]
    #youtuber_data = youtuber_data.drop_duplicates()
    # df2 = df.append(youtuber_data)
    #df2 = df2.drop_duplicates()
    
#     video_data = df1[['video_id', 'title', 'youtuber_id', 'year', 'duration']]
#     df2 = df.append(video_data)
#     df2 = df2.drop_duplicates()
#     video_data = video_data.drop_duplicates(keep='first')
#     video_data = video_data.values.tolist()[0]
    
#     try: 
#         cur.execute(Videos_table_insert, video_data)
#     except psycopg2.Error as e:
#         print(e)

    df = pd.read_json(filepath, lines=True)

    # filter by NextVideo action
    df = df[(df['page'] == 'NextVideo')]

    # convert timestamp column to datetime with 'pd.to_datetime(df["ts"], unit="ms")' 
    df['ts'] = pd.to_datetime(df["ts"], unit="ms")
    # insert time data records to Time_dim table
    
        # write your code here
    df['year'] =  pd.to_datetime(df['ts']).dt.year
    df['month'] =  pd.to_datetime(df['ts']).dt.month
    df['day'] =  pd.to_datetime(df['ts']).dt.day
    df['hour'] =  pd.to_datetime(df['ts']).dt.hour
    df['week'] =  pd.to_datetime(df['ts']).dt.week
    df['weekday'] =  pd.to_datetime(df['ts']).dt.weekday
    df['start_time'] =  pd.to_datetime(df['ts'])
    
    user_data = df[['userId', 'firstName', 'lastName', 'gender', 'location']]
    df2 = df1.append(user_data)
    #df2 = df2.drop_duplicates()

    return df2

def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles 

def process_data(cur, conn, filepath, func):
    """
        This function get all JSON files in given directory by exploring all sub directories, and process all files that were found using the given function.
        Example: if I give it the path to youtube_data directory which resides in data folder of this assignment,
        and func given is process_youtubedata_file it should get all JSON files in this directories and process each file using process_youtubedata_file function. 
        Arguments:
        cur: Database Cursor
        conn: Database
        filepath: location of JSON files
        func: function to process all files in the directory
        Return: None
    """
    df = pd.DataFrame()
    file_list = getListOfFiles(filepath)
    print(len(file_list))
    for file in file_list:
        df = func(cur,conn,file,df)
        
    return df

def main():
    conn = psycopg2.connect("host=127.0.0.1 dbname=youtubedb user=postgres password=ali123ali")
    cur = conn.cursor()
    conn.set_session(autocommit=True)
    df = pd.DataFrame()
    
    #df3 = process_data(cur, conn, filepath='data/youtube_data', func=process_youtubedata_file)
    df3 = process_data(cur, conn, filepath='data/log_data', func=process_youtubedata_file)

    conn.close()
    #print(len(df3))
    return df3


if __name__ == "__main__":
    df = main()
    print(len(df))

30
6820


In [5]:
#df.sort_values(by=['userId'], inplace=True)
df.head(30)
#df.nunique()

,userId,firstName,lastName,gender,location
2,8,Kaylee,Summers,F,"Phoenix-Mesa-Scottsdale, AZ"
4,8,Kaylee,Summers,F,"Phoenix-Mesa-Scottsdale, AZ"
5,8,Kaylee,Summers,F,"Phoenix-Mesa-Scottsdale, AZ"
6,8,Kaylee,Summers,F,"Phoenix-Mesa-Scottsdale, AZ"
7,8,Kaylee,Summers,F,"Phoenix-Mesa-Scottsdale, AZ"
8,8,Kaylee,Summers,F,"Phoenix-Mesa-Scottsdale, AZ"
9,8,Kaylee,Summers,F,"Phoenix-Mesa-Scottsdale, AZ"
10,10,Sylvie,Cruz,F,"Washington-Arlington-Alexandria, DC-VA-MD-WV"
12,26,Ryan,Smith,M,"San Jose-Sunnyvale-Santa Clara, CA"
13,26,Ryan,Smith,M,"San Jose-Sunnyvale-Santa Clara, CA"


In [2]:
# Create a connection
try: 
    t = time.time()
    conn = psycopg2.connect("host=127.0.0.1 user=postgres password=ali123ali")
    print('Time Taken:', time.time()-t)
    
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
# get a cursor    
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)
    print('Time Taken:',time.time()-t)    
    
# set the autocommit to true    
conn.set_session(autocommit=True)

Time Taken: 0.05823040008544922


In [4]:
try: 
    t = time.time()
    cur.execute("create database youtubedb")
    print('Time Taken:',time.time()-t)
    
except psycopg2.Error as e:
    print(e)

database "youtubedb" already exists



### Create_Table_queries.py ###

In [3]:
# DROP TABLES
# Write queries to drop each table, please don't change variable names,
# You should write respective queries against each varibale

Videoplay_table_drop = "DROP TABLE IF EXISTS Videoplay_fact"
Users_table_drop = "DROP TABLE IF EXISTS Users_dim"
Videos_table_drop = "DROP TABLE IF EXISTS Videos_dim"
Youtubers_table_drop = "DROP TABLE IF EXISTS Youtubers_dim"
Time_table_drop = "DROP TABLE IF EXISTS Time_dim"

# CREATE TABLES
# Write queries to create each table, please don't change variable names, you can refer to star schema table
# You should write respective queries against each varibale

Videoplay_table_create = ("""CREATE TABLE IF NOT EXISTS Videoplay_fact (videoplay_id text, start_time TIME, 
                            user_id text, level VARCHAR, video_id text, 
                            youtuber_id text, session_id text, location text, user_agent text, 
                            PRIMARY KEY (videoplay_id), 
                            FOREIGN KEY(start_time) REFERENCES Time_dim(start_time),
                            FOREIGN KEY(user_id) REFERENCES Users_dim(user_id), 
                            FOREIGN KEY(video_id) REFERENCES Videos_dim(video_id),
                            FOREIGN KEY(youtuber_id) REFERENCES Youtubers_dim(youtuber_id));""")

In [4]:
Users_table_create = ("""CREATE TABLE IF NOT EXISTS Users_dim (user_id text, first_name text, 
                        last_name text, gender text, level VARCHAR, 
                        PRIMARY KEY (user_id));""")

Videos_table_create = ("""CREATE TABLE IF NOT EXISTS Videos_dim (Video_id text, title text, 
                        youtuber_id text, year text, duration VARCHAR, 
                        PRIMARY KEY (video_id),
                        FOREIGN KEY(youtuber_id) REFERENCES Youtubers_dim(youtuber_id));""")


Youtubers_table_create = ("""CREATE TABLE IF NOT EXISTS Youtubers_dim (youtuber_id text, name text, 
                        location text, latitude VARCHAR, longitude VARCHAR, 
                        PRIMARY KEY (youtuber_id));""")

Time_table_create = ("""CREATE TABLE IF NOT EXISTS Time_dim (start_time TIME, hour int, 
                        day int, week int, month int, year int, weekday int, 
                        PRIMARY KEY (start_time));""")

In [5]:
# INSERT RECORDS
# Write queries to insert record to each table, please don't change variable names, you can refer to star schema table
# You should write respective queries against each varibale

Youtubers_table_insert = ("""INSERT INTO Youtubers_dim (youtuber_id, name, location, latitude, longitude)
                        VALUES (%s, %s, %s, %s, %s)""")

Videos_table_insert = ("""INSERT INTO Videos_dim (Video_id, title, youtuber_id, year, duration)
                        VALUES (%s, %s, %s, %s, %s)""")

Users_table_insert = ("""INSERT INTO Users_dim (user_id, first_name, last_name, gender, level)
                        VALUES (%s, %s, %s, %s, %s)""")

Time_table_insert = ("""INSERT INTO Time_dim (start_time, hour, day, week, month, year, weekday)
                        VALUES (%s, %s, %s, %s, %s, %s, %s)""")

Videoplay_table_insert = ("""INSERT INTO Youtubers_dim (youtuber_id, name, location, latitude, longitude )
                        VALUES (%s, %s, %s, %s, %s)""")

# youtuber_record = ("""INSERT INTO Youtubers_dim (youtuber_id, name, location, latitude, longitude)
# SELECT youtuber_id, youtuber_name, youtuber_location, youtuber_latitude, youtuber_longitude FROM youtube_data""")

# QUERY LISTS

create_table_queries = [Users_table_create, Time_table_create, Youtubers_table_create, Videos_table_create, Videoplay_table_create]
drop_table_queries = [Videoplay_table_drop, Users_table_drop, Videos_table_drop, Youtubers_table_drop, Time_table_drop]


### Create Tables.py file ###

In [6]:
import psycopg2
# from Create_Table_queries import create_table_queries, drop_table_queries


def create_database():
    '''Creates and connects to youtube  database. Returns cursor and connection to DB'''
    
    # connect to default database
    conn = psycopg2.connect("host=127.0.0.1 user=postgres password = ali123ali")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # create youtube database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS youtubedb")
    cur.execute("CREATE DATABASE Youtubedb WITH ENCODING 'utf8' TEMPLATE template0")

    # close connection to default database
    conn.close()    
    
    # connect to youtubedb database
    conn = psycopg2.connect("host=127.0.0.1 dbname=youtubedb user=postgres password=ali123ali")
    cur = conn.cursor()
    
    return cur, conn


def drop_tables(cur, conn):
    '''Drops all tables created on the database'''
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()


def create_tables(cur, conn):
    '''Created tables defined on the Create_Table_queries script'''
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()


def main():
    """ Function to drop and re create youtube database and all related tables.
        Usage: python Create_Table_queries.py
    """
    cur, conn = create_database()
    
    drop_tables(cur, conn)
    create_tables(cur, conn)

    conn.close()
    
main()


In [44]:
import pandas as pd
filepath = "data\youtube_data\A\A\A\TRAAAAW128F429D538.json"
df = pd.read_json(filepath, orient ='columns')
youtuber_data = df[['youtuber_id', 'youtuber_name', 'youtuber_location', 'youtuber_latitude', 'youtuber_longitude']]

youtuber_data.values.tolist()[0]

    duration             title  year         youtuber_id  youtuber_latitude  \
0  218.93179  I Didn't Mean To     0  ARD7TVE1187B99BFB1                NaN   

  youtuber_location  youtuber_longitude youtuber_name  num_videos  \
0   California - LA                 NaN        Casual           1   

             video_id  
0  SOMZWCG12A8C13C480  


['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

In [27]:
filepath = "data\log_data\\2018\\11\\2018-11-02-events.json"
df = pd.read_json(filepath, lines=True)

# filter by NextVideo action
df = df[(df['page'] == 'NextVideo')]

# convert timestamp column to datetime with 'pd.to_datetime(df["ts"], unit="ms")' 
df['ts'] = pd.to_datetime(df["ts"], unit="ms")
# insert time data records to Time_dim table

    # write your code here
df['year'] =  pd.to_datetime(df['ts']).dt.year
df['month'] =  pd.to_datetime(df['ts']).dt.month
df['day'] =  pd.to_datetime(df['ts']).dt.day
df['hour'] =  pd.to_datetime(df['ts']).dt.hour
df['week'] =  pd.to_datetime(df['ts']).dt.week
df['weekday'] =  pd.to_datetime(df['ts']).dt.weekday
df['start_time'] =  pd.to_datetime(df['ts']).dt.time

videoplay_data = df[['start_time', 'userId', 'level','video', 'youtuber', 'sessionId', 'location','userAgent']]
videoplay_data['videoplay_id'] = list(map(str,list(range(0, len(videoplay_data)))))
videoplay_data = videoplay_data.drop_duplicates(keep='first')
videoplay_data = videoplay_data[['videoplay_id','start_time', 'userId', 'level','video', 'youtuber', 'sessionId', 'location','userAgent']]

df.dtypes

C:\Windows\TEMP/ipykernel_25032/576111745.py:16: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week'] =  pd.to_datetime(df['ts']).dt.week
C:\Windows\TEMP/ipykernel_25032/576111745.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  videoplay_data['videoplay_id'] = list(map(str,list(range(0, len(videoplay_data)))))


youtuber                 object
auth                     object
firstName                object
gender                   object
itemInSession             int64
lastName                 object
length                  float64
level                    object
location                 object
method                   object
page                     object
registration              int64
sessionId                 int64
video                    object
status                    int64
ts               datetime64[ns]
userAgent                object
userId                    int64
year                      int64
month                     int64
day                       int64
hour                      int64
week                      int64
weekday                   int64
start_time               object
dtype: object

In [11]:
import pandas as pd

filepath = "data\log_data\\2018\\11\\2018-11-02-events.json"
#filepath = "data\youtube_data\A\A\A\TRAAAAW128F429D538.json"
df = pd.read_json(filepath, lines=True)
df['start_time'] = pd.to_datetime(df['ts']).dt.time
#df['start_time'] = df['start_time'].astype('str')
#df['videoplay_id'] = range(0, len(df))
df.columns
print(df['start_time'][0])
#df.dtypes
#youtuber_data = df[['youtuber_id', 'youtuber_name', 'youtuber_location', 'youtuber_latitude', 'youtuber_longitude']]

#df.values.tolist()

00:25:41.121934


### Test_tables notebook ### 
Test your tables (Don't change anything in notebook)

In [17]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
# from Create_Table_queries import *

# %load_ext autoreload
# %autoreload 2

def process_youtubedata_file(cur, filepath):
    df = pd.read_json(filepath, lines=True)
    
    """
        This function reads one JSON file and read information of videos and youtuber data and saves into video_data and youtuber_data
        Arguments:
        cur: Database Cursor
        filepath: location of JSON files
        Return: None
    """
    # open JSON file
    youtuber_data = df[['youtuber_id', 'youtuber_name', 'youtuber_location', 'youtuber_latitude', 'youtuber_longitude']]
    #print("Youtuber dataframe\n", youtuber_data)

    # ---------insert youtuber record----------
    # write your code here that reads youtuber data from JSON file and insert it into Youtubers_dim table 
    # write your code here
    
    
    youtuber_record = []
    for pair in youtuber_data.values[:,:]: 
        print("pair:", pair)
        for value in pair:
            print("value:", value)
            for i in value.values() :
                youtuber_record.append(i)
    youtuber_data.append(youtuber_record)
    print("\n\nYoutuber record to be added:", youtuber_record)
    
    
    cur.execute(Youtubers_table_insert, youtuber_record)
    
    # Check for data in the Youtuber_dim Table
#     query = """SELECT * FROM Youtubers_dim LIMIT 5;"""
#     cur.execute(query)
#     row = cur.fetchone()
#     while row:
#         print("Checking data in the table:", row)
#         row = cur.fetchone()
    
    print("Youtuber Dataframe\n", youtuber_data)
    
    
    
    # ---------insert video record--------------
    # write your code here that reads youtube videos data from JSON file and insert it into Videos_dim table 
    # write your code here
    
    video_data = df[['video_id', 'title', 'youtuber_id', 'year', 'duration']]
    print("\n\n\nVideo dataframe\n", video_data)
    
    video_record = []
    for pair in video_data.values[:,:] : 
        for value in pair:
            for i in value.values() :
                video_record.append(i)
    print("\n\nVideo record to be added:", video_record)
    
    cur.execute(Videos_table_insert, video_record)
    
    query = """SELECT * FROM Videos_dim LIMIT 5;"""
    cur.execute(query)
    row = cur.fetchone()
    while row:
        print("Checking data in the table:", row)
        row = cur.fetchone()


In [22]:
def process_log_file(cur, filepath):
    """
        This function reads Log files and reads information of time, user and videoplay data and saves into time, user, videoplay
        Arguments:
        cur: Database Cursor
        filepath: location of Log files
        Return: None
    """

    # open log file
    df = pd.read_json(filepath, lines=True)
    print("Log files data:\n", df)
    # filter by NextVideo action
    
#     df = df[(df['page'] == 'NextVideo')]
#     print("Dataframe:", df)

    # convert timestamp column to datetime with 'pd.to_datetime(df["ts"], unit="ms")'   
    # insert time data records to Time_dim table

    
        # write your code here

    
    
    
    # load user table
    # insert user records into Users_dim table

    
        # write your code here

    
    # insert Videoplay records in Videoplay_fact table

    
        # write your code here

def process_data(cur, conn, filepath, func):
    """
        This function get all JSON files in given directory by exploring all sub directories, and process all files that were found using the given function.
        Example: if I give it the path to youtube_data directory which resides in data folder of this assignment,
        and func given is process_youtubedata_file it should get all JSON files in this directories and process each file using process_youtubedata_file function. 
        Arguments:
        cur: Database Cursor
        conn: Database
        filepath: location of JSON files
        func: function to process all files in the directory
        Return: None
    """   
    func(cur,filepath)
        
def main():
    
    conn = psycopg2.connect("host=127.0.0.1 dbname=youtubedb user=postgres password=ali123ali")
    cur = conn.cursor()

    process_data(cur, conn, filepath='data/youtube_data', func=process_youtubedata_file)
#     process_data(cur, conn, filepath='data/log_data', func=process_log_file)

#     conn.close()

#     conn = psycopg2.connect("host=127.0.0.1 dbname=youtubedb user=postgres password=ali123ali")
#     cur = conn.cursor()

#     for filename in glob.glob('data\youtube_data\A\A\A\*.json'):
#         with open(os.path.join(os.getcwd(), filename), 'r') as f:
#             text = f.read()
#     process_youtubedata_file (cur, text)
    
    
#     for filename in glob.glob('data\log_data\2018\11*.json'):
#         with open(os.path.join(os.getcwd(), filename), 'r') as file_log:
#             text_log = file_log.read()
    
#     process_log_file (cur, text_log)
#     process_log_file (cur, 'data\log_data\2018\11\*.json')
    
#     process_data(cur, conn, filepath='data/youtube_data', func=process_youtubedata_file)
#     process_data(cur, conn, filepath='data/log_data', func=process_log_file)
    
#     conn.close()


if __name__ == "__main__":
    main()

ValueError: Expected object or value

In [47]:
import os
'''
    For the given path, get the List of all files in the directory tree 
'''
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles 

l = getListOfFiles('data\youtube_data')
print(len(l))
print(l[-10:-1])

71
['data\\youtube_data\\A\\B\\C\\TRABCEI128F424C983.json', 'data\\youtube_data\\A\\B\\C\\TRABCFL128F149BB0D.json', 'data\\youtube_data\\A\\B\\C\\TRABCIX128F4265903.json', 'data\\youtube_data\\A\\B\\C\\TRABCKL128F423A778.json', 'data\\youtube_data\\A\\B\\C\\TRABCPZ128F4275C32.json', 'data\\youtube_data\\A\\B\\C\\TRABCRU128F423F449.json', 'data\\youtube_data\\A\\B\\C\\TRABCTK128F934B224.json', 'data\\youtube_data\\A\\B\\C\\TRABCUQ128E0783E2B.json', 'data\\youtube_data\\A\\B\\C\\TRABCXB128F4286BD3.json']
